In [18]:
#Dependencies
import pandas as pd
import hvplot.pandas
import numpy as np
import panel as pn
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats

In [19]:
# Load the data
path = 'Averaged_Cleaned_filtered_data.csv'
Heart_disease_df = pd.read_csv(path)

filtered_df = Heart_disease_df
filtered_df

,US States,Year,Ethnicity,Gender,Heart Disease Type,Age range,Data_Value/100_000 People
0,AK,1999.0,American Indian/Alaska Native,Overall,All heart disease,Ages 35-64 years,84.886667
1,AK,1999.0,American Indian/Alaska Native,Overall,All heart disease,Ages 65 years and older,1433.755556
2,AK,1999.0,American Indian/Alaska Native,Overall,All stroke,Ages 35-64 years,23.786667
3,AK,1999.0,American Indian/Alaska Native,Overall,All stroke,Ages 65 years and older,680.877778
4,AK,1999.0,American Indian/Alaska Native,Overall,Cardiovascular disease (CVD),Ages 35-64 years,87.780000
...,...,...,...,...,...,...,...
22465,IL,2019.0,White,Overall,Cardiovascular disease (CVD),Ages 65 years and older,1561.057143
22466,IL,2019.0,White,Overall,Coronary heart disease (CHD),Ages 35-64 years,66.591429
22467,IL,2019.0,White,Overall,Coronary heart disease (CHD),Ages 65 years and older,577.260000
22468,IL,2019.0,White,Overall,Heart failure,Ages 35-64 years,18.165714


In [26]:
# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()

# Create interactive widgets
ethnicity_selector_dropdown = pn.widgets.Select(name='Select Ethnicity', options=ethnicities)
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])

start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)
age_range_selector = pn.widgets.CheckBoxGroup(name='Select Age Range', options=age_ranges, inline=True)

# Create interactive widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)
age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

In [21]:
# LINE GRAPH (FILTERED BY STATES & RACES & Age Groups)
# Using pandas to filter the required data and then hvplot create an interactive graph with the following features:

# A line graph that plots

# y- axis - Average of the filtered Data_Value/100_000 People
# x - axis - year

# A unique line for each Heart disease type.


# Then has a dropdown box that allows the user to choose which Enthinicity they want to display on the graph. 

# A checkbox that allows the user to choose which states they want to see plotted on the graph.

#     when the ethnicity and states are chosen
#     The orginal dataframe is
#     filtered for all the rows 
#     for the chosen ethnicity and the chosen states (more than one state can be chosen at a time).
#     with this new filtered dataframe, 
#     pandas groups by the heart disease type and year
#     and averages Data_Value/100_000 People.

# This will produce a dataframe that:
#     Has one value for Data_Value/100_000 for each unique heart disease type and year.
#     the enthnicity of all those Data_Value/100_000 datapoints will be the same.

# A line graph that plots

# y- axis -  Data_Value/100_000 People
# x - axis - year

# Then has a dropdown box that allows the user to choose which Enthinicity they want to display on the graph. 
# A checkbox that allows the user to choose which states they want to see plotted on the graph.

# when the race or states has been changed the hvplot graph updates.

In [22]:
# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_dropdown, start_year_selector, end_year_selector, states_selector, age_range_selector)
def update_plot(selected_ethnicity, start_year, end_year, selected_states, selected_age_ranges):
    if not selected_states:
        return "Please select at least one state."
    if not selected_age_ranges:
        return "Please select at least one age range."
    
    # Filter data based on selected ethnicity, states, age ranges, and years
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'] == selected_ethnicity) &
        (filtered_df['US States'].isin(selected_states)) &
        (filtered_df['Age range'].isin(selected_age_ranges)) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    # Group by 'Heart Disease Type' and 'Year', then calculate the mean of 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Heart Disease Type', 'Year'])['Data_Value/100_000 People'].mean().reset_index()
    
    # Create a line plot
    plot = grouped_data.hvplot.line(
        x='Year',
        y='Data_Value/100_000 People',
        by='Heart Disease Type',
        title=f'Heart Disease Types Distribution for {selected_ethnicity}',
        ylabel='Average Data Value per 100,000 People',
        xlabel='Year',
        legend='top',
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_dropdown, pn.Column(start_year_selector, end_year_selector)),
    pn.Column(states_selector, age_range_selector),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Traceback (most recent call last):
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\viewable.py", line 478, in _on_msg
 doc.unhold()
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\bokeh\document\document.py", line 776, in unhold
 self.callbacks.unhold()
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\bokeh\document\callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\bokeh\document\callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\bokeh\util\callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\reactive.py", line 471, in _comm_change
 state._handle_exception(e)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\io\state.py", line 442, in _handle_exception
 raise exception
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\reactive.py", line 469, in _comm_change
 self._schedule_change(doc, comm)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\reactive.py", line 451, in _schedule_change
 self._change_event(doc)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\reactive.py", line 447, in _change_event
 self._process_events(events)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\reactive.py", line 385, in _process_events
 self.param.update(**self_events)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\parameterized.py", line 2278, in update
 restore = self_._update(*args, **kwargs)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\parameterized.py", line 2318, in _update
 self_._batch_call_watchers()
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\parameterized.py", line 2509, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\parameterized.py", line 2471, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\param.py", line 865, in _replace_pane
 new_object = self.eval(self.object)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\panel\param.py", line 824, in eval
 return eval_function_with_deps(function)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\parameterized.py", line 162, in eval_function_with_deps
 return function(*args, **kwargs)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\param\depends.py", line 41, in _depends
 return func(*args, **kw)
 File "C:\Users\morga\AppData\Local\Temp\ipykernel_13068\2593659449.py", line 13, in update_plot
 (filtered_df['Age range'].isin(selected_age_ranges)) &
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\pandas\core\series.py", line 5350, in isin
 result = algorithms.isin(self._values, values)
 File "C:\Users\morga\anaconda3\envs\dev\lib\site-packages\pandas\core\algorithms.py", line 477, in isin
 raise TypeError(
TypeError: only list-like objects are allowed to be passed to isin(), you passed a `str`

Column
    [0] Row
        [0] Select(options=['American Indian/Alaska N...], value='American Indian/Alaska N...)
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Column
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
        [1] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
    [2] ParamFunction(function, _pane=Markdown, defer_load=False)

In [ ]:
# Boxplot (FILTERED BY STATES & RACES & Age groups)
# Using pandas to filter the required data and then hvplot create an interactive graph with the following features:

# a boxplot for each heart disease type that shows the distribution of data across all the years.

# Then checkbox that allows the user to choose which Enthinicities they want to display on the graph.
# Drop down box that allows users to choose the starting year and the ending year of the data.
# A checkbox that allows the user to choose which states they want to see plotted on the graph.
# A checkbox that allows the user to choose which age group (Age range) they want to see plotted on the graph.

#     when the ethnicity, age group, start year, end year, and states are chosen
#     The orginal dataframe is
#     filtered for all the rows 
#     for the chosen ethnicity (more than one can be chosen for each ethnicity), age range (more than one age range can be chosen at time) and states (more than one state can be chosen at a time) and, 
#     with year between and including the start year and end years chosen.
#     With this new filtered dataframe, 
#     pandas groups by the heart disease type age range and ethinicity.

# Then plots a boxplot for each heart disease type, age range and ethnicity chosen.

In [29]:
# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_ranges):
    if not selected_states:
        return "Please select at least one state."
    if not selected_age_ranges:
        return "Please select at least one age range."
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Filter data based on selected ethnicity, states, age ranges, and years
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['US States'].isin(selected_states)) &
        (filtered_df['Age range'].isin(selected_age_ranges)) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    # Create a boxplot
    plot = filtered_data.hvplot.box(
        y='Data_Value/100_000 People',
        by=['Heart Disease Type', 'Age range', 'Ethnicity'],
        title=f'Heart Disease Types Distribution for {", ".join(selected_ethnicities)}',
        xlabel='Heart Disease Type, Age Range, Ethnicity',
        ylabel='Data Value per 100,000 People',
        legend='top_right',
        rot=45,
        xrotation = 45,
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
        
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Column(states_selector, age_range_selector),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Column
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...], value=['CA'])
        [1] CheckBoxGroup(inline=True, name='Select Age Range', options=['Ages 35-64 years', ...], value=['Ages 35-64 years'])
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [8]:
# Bargraph (FILTERED BY STATES & RACES & Age groups)
# Using pandas to filter the required data and then hvplot create an interactive graph with the following features:

# Drop down box that allows users to choose the year of the data they want to look at.

# A checkbox that allows users to choose which heart disease type they want to look at.

# A checkbox that allows users to choose which ethnicity they want to look at.


#     when the year, heart disease types and ethnicities are chosen
#     The orginal dataframe is
#     filtered for all the rows 
#     for the chosen ethnicities (more than one can be chosen for each ethnicity) and chosen heart disease types and year,
#     With this new filtered dataframe, 

# Then plots a bar graph graph,
#  that shows a bar for total Data_Value/100_000 People for each chosen ethnicity,
#    for each chosen ethnicity there will be one bar that represents Data_Value/100_000 People for each chosen heart disease type.type.

In [28]:
heart_disease_df = Heart_disease_df

# Unique values for dropdowns and checkboxes
years = sorted(Heart_disease_df['Year'].unique().astype(int).tolist())
disease_types = Heart_disease_df['Heart Disease Type'].unique().tolist()
ethnicities = Heart_disease_df['Ethnicity'].unique().tolist()
age_ranges = Heart_disease_df['Age range'].unique().tolist()

# Create interactive widgets
year_selector = pn.widgets.Select(name='Select Year', options=years, value=years[0])
disease_selector = pn.widgets.CheckBoxGroup(name='Select Disease Type', options=disease_types, value=[disease_types[0]], inline=True)
ethnicity_selector = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]], inline=True)
age_range_selector = pn.widgets.CheckBoxGroup(name='Select Age Range', options=age_ranges, value=[age_ranges[0]], inline=True)

# Function to update the plot based on widget values
@pn.depends(year_selector, disease_selector, ethnicity_selector, age_range_selector)
def update_plot(selected_year, selected_diseases, selected_ethnicities, selected_age_ranges):
    # Filter data based on selected year, diseases, ethnicities, and age ranges
    filtered_data = heart_disease_df[
        (heart_disease_df['Year'] == selected_year) &
        (heart_disease_df['Heart Disease Type'].isin(selected_diseases)) &
        (heart_disease_df['Ethnicity'].isin(selected_ethnicities)) &
        (heart_disease_df['Age range'].isin(selected_age_ranges))
    ]
    
    # Group by 'Ethnicity', 'Heart Disease Type', and 'Age range', then sum the 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Ethnicity', 'Heart Disease Type', 'Age range'])['Data_Value/100_000 People'].sum().reset_index()
    
    # Create a bar plot
    plot = grouped_data.hvplot.bar(
        x='Ethnicity',
        y='Data_Value/100_000 People',
        by=['Heart Disease Type', 'Age range'],
        title=f'Total Data Value per 100,000 People by Ethnicity, Disease Type, and Age Range for {selected_year}',
        xlabel='Ethnicity',
        ylabel='Total Data Value per 100,000 People',
        rot=45,
        width=800,
        height=600
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    year_selector,
    pn.Column(disease_selector, ethnicity_selector, age_range_selector),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Select(options=[1999, 2000, 2001, ...], value=1999)
    [1] Column
        [0] CheckBoxGroup(inline=True, name='Select Disease Type', options=['All heart disease', ...], value=['All heart disease'])
        [1] CheckBoxGroup(inline=True, name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [2] CheckBoxGroup(inline=True, name='Select Age Range', options=['Ages 35-64 years', ...], value=['Ages 35-64 years'])
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [24]:
import panel as pn
import hvplot.pandas
import pandas as pd

# Assuming the filtered_df DataFrame is already defined with the necessary data.

# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

# Create interactive widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)
age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector, heart_disease_type_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_range, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['US States'].isin(selected_states)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    # Group by 'Ethnicity' and 'Year', then calculate the mean of 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Ethnicity', 'Year'])['Data_Value/100_000 People'].mean().reset_index()
    
    # Create a line plot
    plot = grouped_data.hvplot.line(
        x='Year',
        y='Data_Value/100_000 People',
        by='Ethnicity',
        title=f'{selected_heart_disease_type} Mortality Rates by Ethnicity',
        ylabel='Average Data Value per 100,000 People',
        xlabel='Year',
        legend='top',
        width=800,
        height=600
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Row(states_selector),
    pn.Row(pn.Column(age_range_selector, heart_disease_type_selector)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [2] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

In [25]:
import panel as pn
import hvplot.pandas
import pandas as pd

# Assuming the filtered_df DataFrame is already defined with the necessary data.

# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

# Create interactive widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=['All States'] + states, inline=True)
age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector, heart_disease_type_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_range, selected_heart_disease_type):
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Determine if 'All States' is selected
    all_states_selected = 'All States' in selected_states
    if not selected_states or (not all_states_selected and not any(state in selected_states for state in states)):
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    if not all_states_selected:
        filtered_data = filtered_data[filtered_df['US States'].isin(selected_states)]
    
    # Group by 'Ethnicity' and 'Year', then calculate the mean of 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Ethnicity', 'Year'])['Data_Value/100_000 People'].mean().reset_index()
    
    # Create a line plot
    plot = grouped_data.hvplot.line(
        x='Year',
        y='Data_Value/100_000 People',
        by='Ethnicity',
        title=f'{selected_heart_disease_type} Mortality Rates by Ethnicity',
        ylabel='Average Data Value per 100,000 People',
        xlabel='Year',
        legend='top',
        width=800,
        height=600
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Row(states_selector),
    pn.Row(pn.Column(age_range_selector, heart_disease_type_selector)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['All States', ...])
    [2] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

In [30]:
import panel as pn
import hvplot.pandas
import pandas as pd

# Assuming the filtered_df DataFrame is already defined with the necessary data.

# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

# Create interactive widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=['All States'] + states, inline=True)
age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_checkbox, start_year_selector, end_year_selector, states_selector, age_range_selector, heart_disease_type_selector)
def update_plot(selected_ethnicities, start_year, end_year, selected_states, selected_age_range, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    # Determine if 'All States' is selected
    all_states_selected = 'All States' in selected_states
    if not all_states_selected and not any(state in selected_states for state in states):
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    if not all_states_selected:
        filtered_data = filtered_data[filtered_df['US States'].isin(selected_states)]
    
    # Create a boxplot
    plot = filtered_data.hvplot.box(
        y='Data_Value/100_000 People',
        by=['Heart Disease Type', 'Age range', 'Ethnicity'],
        title=f'{selected_heart_disease_type} Mortality Rates Distribution by Ethnicity and Age Range',
        xlabel='Heart Disease Type, Age Range, Ethnicity',
        ylabel='Data Value per 100,000 People',
        legend='top_right',
        rot=45,
        xrotation=45,
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_checkbox, pn.Column(start_year_selector, end_year_selector)),
    pn.Row(states_selector),
    pn.Row(pn.Column(age_range_selector, heart_disease_type_selector)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] Column
            [0] Select(options=[1999, 2000, 2001, ...], value=1999)
            [1] Select(options=[1999, 2000, 2001, ...], value=2019)
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['All States', ...])
    [2] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)

In [31]:
import panel as pn
import hvplot.pandas
import pandas as pd

# Assuming the filtered_df DataFrame is already defined with the necessary data.

# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

# Create interactive widgets
ethnicity_selector_dropdown = pn.widgets.Select(name='Select Ethnicity', options=ethnicities, value=ethnicities[0])
states_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)
age_range_selector_dropdown = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
heart_disease_type_selector_dropdown = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])

# Function to update the plot based on widget values
@pn.depends(ethnicity_selector_dropdown, age_range_selector_dropdown, states_selector_checkbox, heart_disease_type_selector_dropdown)
def update_plot(selected_ethnicity, selected_age_range, selected_states, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'] == selected_ethnicity) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['US States'].isin(selected_states))
    ]
    
    if filtered_data.empty:
        return "No data available for the selected criteria."
    
    # Pivot the data to create a correlation matrix
    pivot_table = filtered_data.pivot_table(
        values='Data_Value/100_000 People', 
        index='Year', 
        columns='US States'
    )
    
    # Calculate the correlation matrix
    correlation_matrix = pivot_table.corr()

    # Create a heatmap
    heatmap = correlation_matrix.hvplot.heatmap(
        title=f'Correlation Matrix for {selected_heart_disease_type} Mortality Rates in {selected_ethnicity} ({selected_age_range})',
        cmap='viridis',
        width=800,
        height=600,
        rot=45,
        xrotation=45
    ).opts(colorbar=True)

    return heatmap

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    pn.Row(ethnicity_selector_dropdown, pn.Column(age_range_selector_dropdown, heart_disease_type_selector_dropdown)),
    pn.Row(states_selector_checkbox),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Row
        [0] Select(options=['American Indian/Alaska N...], value='American Indian/Alaska N...)
        [1] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [1] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [2] ParamFunction(function, _pane=Markdown, defer_load=False)